# 基于Transformers的命名实体识别

In [2]:
model_path="/home/model/chinese-macbert-base"

## Step1 导入相关包

In [3]:
import evaluate
from datasets import load_dataset
from transformers import AutoTokenizer, AutoModelForTokenClassification, TrainingArguments, Trainer, DataCollatorForTokenClassification
from seqeval.metrics import classification_report


## Step2 加载数据集

In [4]:
# 如果可以联网，直接使用load_dataset进行加载
ner_datasets = load_dataset("peoples_daily_ner")
# 如果无法联网，则使用下面的方式加载数据集
# from datasets import DatasetDict
# ner_datasets = DatasetDict.load_from_disk("ner_data")
ner_datasets

Using the latest cached version of the module from /home/wiseworker/.cache/huggingface/modules/datasets_modules/datasets/peoples_daily_ner/594461a1b34f61af9346123a420b9ea40f15c0e835562053bf025cef188477f5 (last modified on Wed Aug 16 15:42:39 2023) since it couldn't be found locally at peoples_daily_ner., or remotely on the Hugging Face Hub.


DatasetDict({
    train: Dataset({
        features: ['id', 'tokens', 'ner_tags'],
        num_rows: 20865
    })
    validation: Dataset({
        features: ['id', 'tokens', 'ner_tags'],
        num_rows: 2319
    })
    test: Dataset({
        features: ['id', 'tokens', 'ner_tags'],
        num_rows: 4637
    })
})

In [5]:
print(ner_datasets["train"][0])

{'id': '0', 'tokens': ['海', '钓', '比', '赛', '地', '点', '在', '厦', '门', '与', '金', '门', '之', '间', '的', '海', '域', '。'], 'ner_tags': [0, 0, 0, 0, 0, 0, 0, 5, 6, 0, 5, 6, 0, 0, 0, 0, 0, 0]}


In [6]:
label_list = ner_datasets["train"].features["ner_tags"].feature.names
print(label_list)

['O', 'B-PER', 'I-PER', 'B-ORG', 'I-ORG', 'B-LOC', 'I-LOC']


## Step3 数据集预处理

In [8]:
tokenizer = AutoTokenizer.from_pretrained(model_path)

In [9]:
# 借助word_ids 实现标签映射
def process_function(examples):
    tokenized_exmaples = tokenizer(examples["tokens"], max_length=128, truncation=True, is_split_into_words=True)
    labels = []
    for i, label in enumerate(examples["ner_tags"]):
        word_ids = tokenized_exmaples.word_ids(batch_index=i)
        label_ids = []
        for word_id in word_ids:
            if word_id is None:
                label_ids.append(-100)
            else:
                label_ids.append(label[word_id])
        labels.append(label_ids)
    tokenized_exmaples["labels"] = labels
    return tokenized_exmaples

In [10]:
tokenized_datasets = ner_datasets.map(process_function, batched=True)
tokenized_datasets

Map:   0%|          | 0/2319 [00:00<?, ? examples/s]

DatasetDict({
    train: Dataset({
        features: ['id', 'tokens', 'ner_tags', 'input_ids', 'token_type_ids', 'attention_mask', 'labels'],
        num_rows: 20865
    })
    validation: Dataset({
        features: ['id', 'tokens', 'ner_tags', 'input_ids', 'token_type_ids', 'attention_mask', 'labels'],
        num_rows: 2319
    })
    test: Dataset({
        features: ['id', 'tokens', 'ner_tags', 'input_ids', 'token_type_ids', 'attention_mask', 'labels'],
        num_rows: 4637
    })
})

In [11]:
print(tokenized_datasets["train"])

Dataset({
    features: ['id', 'tokens', 'ner_tags', 'input_ids', 'token_type_ids', 'attention_mask', 'labels'],
    num_rows: 20865
})


## Step4 创建模型

In [12]:
# 对于所有的非二分类任务，切记要指定num_labels，否则就会device错误
model = AutoModelForTokenClassification.from_pretrained(model_path, num_labels=len(label_list))

Some weights of BertForTokenClassification were not initialized from the model checkpoint at /data1/model/chinese-macbert-base and are newly initialized: ['classifier.weight', 'classifier.bias']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


## Step5 创建评估函数

In [13]:
# 这里方便大家加载，替换成了本地的加载方式，无需额外下载
seqeval = evaluate.load("seqeval_metric.py")
seqeval

EvaluationModule(name: "seqeval", module_type: "metric", features: {'predictions': Sequence(feature=Value(dtype='string', id='label'), length=-1, id='sequence'), 'references': Sequence(feature=Value(dtype='string', id='label'), length=-1, id='sequence')}, usage: """
Produces labelling scores along with its sufficient statistics
from a source against one or more references.

Args:
    predictions: List of List of predicted labels (Estimated targets as returned by a tagger)
    references: List of List of reference labels (Ground truth (correct) target values)
    suffix: True if the IOB prefix is after type, False otherwise. default: False
    scheme: Specify target tagging scheme. Should be one of ["IOB1", "IOB2", "IOE1", "IOE2", "IOBES", "BILOU"].
        default: None
    mode: Whether to count correct entity labels with incorrect I/B tags as true positives or not.
        If you want to only count exact matches, pass mode="strict". default: None.
    sample_weight: Array-like of sha

In [14]:
import numpy as np

def eval_metric(pred):
    predictions, labels = pred
    predictions = np.argmax(predictions, axis=-1)

    # 将id转换为原始的字符串类型的标签
    true_predictions = [
        [label_list[p] for p, l in zip(prediction, label) if l != -100]
        for prediction, label in zip(predictions, labels) 
    ]

    true_labels = [
        [label_list[l] for p, l in zip(prediction, label) if l != -100]
        for prediction, label in zip(predictions, labels) 
    ]

    result = seqeval.compute(predictions=true_predictions, references=true_labels, mode="strict", scheme="IOB2")

    # report = classification_report(true_labels, true_predictions)
    # return report

    return {
        "f1": result["overall_f1"],
        "recall": result["overall_recall"],
        "accuracy": result["overall_accuracy"]
    }
    

## Step6 配置训练参数

In [15]:
args = TrainingArguments(
    output_dir="models_for_ner",
    per_device_train_batch_size=64,
    per_device_eval_batch_size=128,
    evaluation_strategy="epoch",
    save_strategy="epoch",
    logging_steps=50,
    num_train_epochs=3
)

## Step7 创建训练器

In [16]:
trainer = Trainer(
    model=model,
    args=args,
    train_dataset=tokenized_datasets["train"],
    eval_dataset=tokenized_datasets["validation"],
    compute_metrics=eval_metric,
    data_collator=DataCollatorForTokenClassification(tokenizer=tokenizer)
)

## Step8 模型训练

In [17]:
trainer.train()

/root/anaconda3/envs/bert_crf/lib/python3.8/site-packages/transformers/optimization.py:411: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(
You're using a BertTokenizerFast tokenizer. Please note that with a fast tokenizer, using the `__call__` method is faster than using a method to encode the text followed by a call to the `pad` method to get a padded encoding.


Epoch,Training Loss,Validation Loss,F1,Recall,Accuracy
1,0.029200,0.019827,0.944015,0.936013,0.993779
2,0.013000,0.017800,0.952001,0.946404,0.994630
3,0.005900,0.017741,0.956712,0.954881,0.995163


TrainOutput(global_step=981, training_loss=0.030959200603881742, metrics={'train_runtime': 337.1913, 'train_samples_per_second': 185.636, 'train_steps_per_second': 2.909, 'total_flos': 3915383795505534.0, 'train_loss': 0.030959200603881742, 'epoch': 3.0})

In [18]:
trainer.evaluate(eval_dataset=tokenized_datasets["test"])

{'eval_loss': 0.023305043578147888,
 'eval_f1': 0.9483832094798152,
 'eval_recall': 0.950738255033557,
 'eval_accuracy': 0.9936748981551398,
 'eval_runtime': 9.956,
 'eval_samples_per_second': 465.75,
 'eval_steps_per_second': 3.716,
 'epoch': 3.0}

## Step9 模型预测

In [24]:
from transformers import pipeline


In [25]:
# 使用pipeline进行推理，要指定id2label
model.config.id2label = {idx: label for idx, label in enumerate(label_list)}
model.config.label2id = {label:idx for idx, label in enumerate(label_list)}
model.config

BertConfig {
  "_name_or_path": "/data1/model/chinese-macbert-base",
  "architectures": [
    "BertForTokenClassification"
  ],
  "attention_probs_dropout_prob": 0.1,
  "classifier_dropout": null,
  "directionality": "bidi",
  "gradient_checkpointing": false,
  "hidden_act": "gelu",
  "hidden_dropout_prob": 0.1,
  "hidden_size": 768,
  "id2label": {
    "0": "O",
    "1": "B-PER",
    "2": "I-PER",
    "3": "B-ORG",
    "4": "I-ORG",
    "5": "B-LOC",
    "6": "I-LOC"
  },
  "initializer_range": 0.02,
  "intermediate_size": 3072,
  "label2id": {
    "B-LOC": 5,
    "B-ORG": 3,
    "B-PER": 1,
    "I-LOC": 6,
    "I-ORG": 4,
    "I-PER": 2,
    "O": 0
  },
  "layer_norm_eps": 1e-12,
  "max_position_embeddings": 512,
  "model_type": "bert",
  "num_attention_heads": 12,
  "num_hidden_layers": 12,
  "pad_token_id": 0,
  "pooler_fc_size": 768,
  "pooler_num_attention_heads": 12,
  "pooler_num_fc_layers": 3,
  "pooler_size_per_head": 128,
  "pooler_type": "first_token_transform",
  "position

In [26]:
# 如果模型是基于GPU训练的，那么推理时要指定device
# 对于NER任务，可以指定aggregation_strategy为simple，得到具体的实体的结果，而不是token的结果
# model = AutoModelForTokenClassification.from_pretrained("./models_for_ner/checkpoint-66")
ner_pipe = pipeline("ner", model=model, tokenizer=tokenizer, device=0,aggregation_strategy='simple')

In [27]:
res = ner_pipe("2013年，姚明当选为第十二届全国政协委员")
res

[{'entity_group': 'PER',
  'score': 0.9994434,
  'word': '姚 明',
  'start': 6,
  'end': 8},
 {'entity_group': 'ORG',
  'score': 0.99223536,
  'word': '第 十 二 届 全 国 政 协',
  'start': 11,
  'end': 19}]

In [28]:
# 根据start和end取实际的结果
ner_result = {}
x = "2013年，姚明当选为第十二届全国政协委员"
for r in res:
    if r["entity_group"] not in ner_result:
        ner_result[r["entity_group"]] = []
    ner_result[r["entity_group"]].append(x[r["start"]: r["end"]])

ner_result

{'PER': ['姚明'], 'ORG': ['第十二届全国政协']}